In [6]:
import pandas as pd
import numpy as np
import os
import pycountry

In [7]:
RAW_FOLDER = "raw_data"
OUTPUT_CSV = "final_panel_2010_2024.csv"
OUTPUT_XLSX = "final_panel_2010_2024.xlsx"

YEAR_START, YEAR_END = 2010, 2024

In [8]:
VARIABLE_NAMES = {
    "poverty_nat": "Poverty headcount ratio at national poverty lines",
    "poverty_83": "Poverty headcount ratio at 8.30",
    "emp_share": "Employment by industry",
    "food_price": "Food price index",
    "ag_va": "Gross Agri Production Value Added",
    "wage_food": "Wage in the food industry",
    "gdp_pc": "GDP per capita (current US$)",
    "gdp_growth": "GDP per capita growth (annual %)",
    "inflation": "Inflation, GDP deflator (annual %)",
    "enrollment": "Secondary enrollment",
    "transfers": "Subsidies and other transfers",
    "urban": "Urban population (% of total population)",
    "precip": "Average precipitation in depth (mm per year)",
    "lpi": "Logistic performance index",
    "region": "region",
    "income_class": "income_class"
}

In [27]:
poverty_nat = pd.read_excel('raw_data\\Poverty headcount ratio at national poverty lines.xls')

In [ ]:
country_list = poverty_nat['Country Name'].unique()
print(f"Number of countries in master list: {len(country_list)}")
print("First 5 countries:", country_list[:5])


In [292]:
poverty_nat.to_excel('preclean_data\\Poverty headcount ratio at national poverty lines.xlsx')

In [15]:
def filter_countries(df, country_column='Country Name'):
    """
    Filters a dataframe to keep only countries in the country_list
    """
    # Check if the specified country column exists
    if country_column not in df.columns:
        print(f"Warning: Column '{country_column}' not found. Available columns: {df.columns.tolist()}")
        return df
    
    # Filter the dataframe
    filtered_df = df[df[country_column].isin(country_list)].copy()
    
    print(f"Original shape: {df.shape}, Filtered shape: {filtered_df.shape}")
    print(f"Countries kept: {filtered_df[country_column].nunique()}")
    
    return filtered_df

In [ ]:
poverty_83 = pd.read_excel('raw_data\\Poverty headcount ratio at 8.30.xlsx',
                           na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(poverty_83.columns.to_list())

In [ ]:
poverty_83 = filter_countries(poverty_83, 'Country Name')

In [ ]:
poverty_83 = poverty_83.rename(columns={'2010 [YR2010]': '2010',
                           '2011 [YR2011]': '2011',
                           '2012 [YR2012]': '2012',
                           '2013 [YR2013]': '2013',
                           '2014 [YR2014]': '2014',
                           '2015 [YR2015]': '2015',
                           '2016 [YR2016]': '2016',
                           '2017 [YR2017]': '2017',
                           '2018 [YR2018]': '2018',
                           '2019 [YR2019]': '2019',
                           '2020 [YR2020]': '2020',
                           '2021 [YR2021]': '2021',
                           '2022 [YR2022]': '2022',
                           '2023 [YR2023]': '2023',
                           '2024 [YR2024]': '2024'})
poverty_83.head()

In [97]:
poverty_83.to_excel('preclean_data\\Poverty headcount ratio at 8.30.xlsx', index=False)

In [ ]:
ag_va = pd.read_excel('raw_data\\Share of Food VA in GDP.xls', 
                      na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(ag_va.columns.to_list())

In [ ]:
ag_va.head()

In [ ]:
ag_va = ag_va.pivot(index=['Country Name', 'Indicator'], columns='Year').reset_index()
ag_va.head()


In [ ]:
ag_va.columns

In [302]:
new_columns = ['Country Name', 'Indicator', '2010', '2011', '2012', '2013', '2014', '2015',
               '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

In [303]:
ag_va.columns = new_columns

In [ ]:
ag_va.head()

In [306]:
ag_va.to_excel('preclean_data\\Share of Food VA in GDP.xlsx', index=False)

In [ ]:
wage_food = pd.read_excel('raw_data\\Wage in the food industry.xlsx',
                          na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(wage_food.columns. to_list())

In [ ]:
wage_food.head()

In [156]:
wage_food = wage_food[wage_food['sex.label'] == 'Total']

In [157]:
wage_food = wage_food[wage_food['classif1.label'] == 'Economic activity (Broad sector): Agriculture']

In [ ]:
wage_food = wage_food[wage_food['classif2.label'] == 'Currency: U.S. dollars']
wage_food.head()

In [159]:
wage_food = wage_food[['ref_area.label', 'indicator.label', 'classif1.label', 'classif2.label', 'time', 'obs_value']]

In [ ]:
wage_food = wage_food.rename(columns={'ref_area.label': 'Country Name',
                          'indicator.label': 'Indicator',            
                          'classif1.label': 'Sector',
                          'classif2.label': 'Currency',
                          'time': 'date',
                          'obs_value': 'value'})
wage_food.head()

In [161]:
wage_food.to_excel('preclean_data\\Wage in the food industry_long.xlsx', index=False)

In [162]:
wage_food = wage_food.pivot(index=['Country Name', 'Indicator', 'Sector', 'Currency'], columns='date')

In [ ]:
wage_food = wage_food.reset_index()
wage_food.columns

In [164]:
# Create new column names
new_columns = []
for col in wage_food.columns:
    if col[0] in ['Country Name', 'Indicator', 'Sector', 'Currency']:
        # Keep the names from the first level
        new_columns.append(col[0])
    else:
        # For the 'value' columns, use the year (second level)
        new_columns.append(str(col[1]))  # Convert year to string

In [165]:
wage_food.columns = new_columns

In [ ]:
wage_food.head()

In [167]:
wage_food.to_excel('preclean_data\\Wage in the food industry_wide.xlsx', index=False)

In [ ]:
emp_share = pd.read_excel('raw_data\\Employment by industry.xlsx',
                          na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(emp_share.columns.to_list())

In [ ]:
emp_share.head()

In [ ]:
emp_share.shape

In [ ]:
emp_share = emp_share[emp_share['sex.label'] == 'Total']
emp_share.shape

In [ ]:
emp_share = emp_share[(emp_share['classif1.label'] == 'Economic activity (Broad sector): Total') | (emp_share['classif1.label'] == 'Economic activity (Broad sector): Agriculture')]
emp_share.shape

In [204]:
emp_share = emp_share[['ref_area.label', 'indicator.label', 'classif1.label', 'time', 'obs_value']]

In [ ]:
emp_share.head()

In [ ]:
emp_share = emp_share.pivot(index=['ref_area.label', 'indicator.label', 'time'], columns='classif1.label').reset_index()
emp_share.head()

In [208]:
# Create new column names
new_columns = ['ref_area.label', 'indicator.label', 'time', 'Economic activity (Broad sector): Agriculture', 'Economic activity (Broad sector): Total']

In [209]:
emp_share.columns = new_columns

In [210]:
emp_share['emp_share'] = emp_share['Economic activity (Broad sector): Agriculture']/emp_share['Economic activity (Broad sector): Total']

In [211]:
emp_share = emp_share[['ref_area.label', 'indicator.label', 'time', 'emp_share']]
emp_share = emp_share.rename(columns={'ref_area.label': 'Country Name',
                                      'indicator.label': 'Indicator',
                                      'time': 'date'})

In [ ]:
emp_share = emp_share.pivot(index=['Country Name', 'Indicator'], columns='date').reset_index()
emp_share.columns

In [216]:
emp_share.columns = ['Country Name', 'Indicator'] + [str(year) for year in range(1947, 2025)]

In [ ]:
emp_share.head()

In [218]:
emp_share.to_excel('preclean_data\\Employment by industry.xlsx', index=False)

In [219]:
food_price = pd.read_excel('raw_data\\Food price index.xls',
                           na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(food_price.columns.to_list())

['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', '2010 [2010]', '2010 [2010].1', '2010 [2010].2', '2011 [2011]', '2011 [2011].1', '2011 [2011].2', '2012 [2012]', '2012 [2012].1', '2012 [2012].2', '2013 [2013]', '2013 [2013].1', '2013 [2013].2', '2014 [2014]', '2014 [2014].1', '2014 [2014].2', '2015 [2015]', '2015 [2015].1', '2015 [2015].2', '2016 [2016]', '2016 [2016].1', '2016 [2016].2', '2017 [2017]', '2017 [2017].1', '2017 [2017].2', '2018 [2018]', '2018 [2018].1', '2018 [2018].2', '2019 [2019]', '2019 [2019].1', '2019 [2019].2', '2020 [2020]', '2020 [2020].1', '2020 [2020].2', '2021 [2021]', '2021 [2021].1', '2021 [2021].2', '2022 [2022]', '2022 [2022].1', '2022 [2022].2', '2023 [2023]', '2023 [2023].1', '2023 [2023].2', '2024 [2024]', '2024 [2024].1', '2024 [2024].2']


In [ ]:
food_price.head()

In [ ]:
food_price = food_price.rename(columns={'Unnamed: 0': 'Country Name',
                      'Unnamed: 1': 'Currency',
                      'Unnamed: 2': 'Indicator',
                      '2010 [2010].1': '2010',
                      '2011 [2011].1': '2011',
                      '2012 [2012].1': '2012',
                      '2013 [2013].1': '2013',
                      '2014 [2014].1': '2014',
                      '2015 [2015].1': '2015',
                      '2016 [2016].1': '2016',
                      '2017 [2017].1': '2017',
                      '2018 [2018].1': '2018',
                      '2019 [2019].1': '2019',
                      '2020 [2020].1': '2020',
                      '2021 [2021].1': '2021',
                      '2022 [2022].1': '2022',
                      '2023 [2023].1': '2023',
                      '2024 [2024].1': '2024'})
food_price.head()

In [222]:
food_price = food_price[['Country Name', 'Currency', 'Indicator', '2010', 
               '2011', '2012', '2013', '2014', '2015', '2016', '2017',
               '2018', '2019', '2020', '2021', '2022', '2023', '2024']]

In [ ]:
food_price = food_price.drop(0)
food_price.head()

In [227]:
food_price = pd.melt(food_price, id_vars=['Country Name', 'Currency', 'Indicator'], var_name='year', value_name='value')

In [ ]:
food_price = food_price.pivot(index=['Country Name', 'Currency', 'year'], columns='Indicator').reset_index()
food_price.columns

In [231]:
new_columns = ['Country Name', 'Currency', 'Year', 'GDP Deflator', 'Agri Deflator']

In [232]:
food_price.columns = new_columns

In [234]:
food_price['Food Price'] = food_price['Agri Deflator']/food_price['GDP Deflator']

In [236]:
food_price = food_price[['Country Name', 'Year', 'Food Price']]

In [ ]:
food_price= food_price.pivot(index='Country Name', columns='Year').reset_index()
food_price.columns

In [239]:
new_columns = ['Country Name'] + [str(year) for year in range(2010, 2025)]
food_price.columns = new_columns

In [241]:
food_price['Indicator'] = 'Relative price of staple foods'

In [ ]:
food_price = food_price[['Country Name', 'Indicator', '2010', '2021', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 
                         '2020', '2021', '2022', '2023', '2024']]
food_price.head()

In [ ]:
food_price['Country Name'] = food_price['Country Name'].str.replace(r'\s*\[\d+\]\s*', '', regex=True)
food_price.head()

In [245]:
food_price.to_excel('preclean_data\\Food price index.xlsx', index=False)

In [246]:
gdp_pc = pd.read_excel('raw_data\\GDP per capita (current US$).xls',
                       na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(gdp_pc.columns.to_list())

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [ ]:
gdp_pc.head()

In [ ]:
gdp_pc = filter_countries(gdp_pc, 'Country Name')
gdp_pc.head()

In [250]:
gdp_pc.to_excel('preclean_data\\GDP per capita (current US$).xlsx', index=False)

In [ ]:
gdp_growth = pd.read_excel('raw_data\\GDP per capita growth (annual %).xls',
                           na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(gdp_growth.columns.to_list())

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [ ]:
gdp_growth = filter_countries(gdp_growth, 'Country Name')
gdp_growth.head()

In [253]:
gdp_growth.to_excel('preclean_data\\GDP per capita growth (annual %).xlsx', index=False)

In [ ]:
inflation = pd.read_excel('raw_data\\Inflation, GDP deflator (annual %).xls',
                          na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(inflation.columns.to_list())

In [ ]:
inflation = filter_countries(inflation, 'Country Name')
inflation.head()

In [256]:
inflation.to_excel('preclean_data\\Inflation, GDP deflator (annual %).xlsx', index=False)

In [ ]:
enrollment = pd.read_excel('raw_data\\Secondary enrollment.xlsx',
                            na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(enrollment.columns.to_list())

In [ ]:
enrollment = filter_countries(enrollment, 'Country Name')
enrollment.head()

In [ ]:
enrollment = enrollment.rename(columns={'2010 [YR2010]': '2010',
                           '2011 [YR2011]': '2011',
                           '2012 [YR2012]': '2012',
                           '2013 [YR2013]': '2013',
                           '2014 [YR2014]': '2014',
                           '2015 [YR2015]': '2015',
                           '2016 [YR2016]': '2016',
                           '2017 [YR2017]': '2017',
                           '2018 [YR2018]': '2018',
                           '2019 [YR2019]': '2019',
                           '2020 [YR2020]': '2020',
                           '2021 [YR2021]': '2021',
                           '2022 [YR2022]': '2022',
                           '2023 [YR2023]': '2023',
                           '2024 [YR2024]': '2024'})
enrollment.head()

In [265]:
enrollment.to_excel('preclean_data\\Secondary enrollment.xlsx', index=False)

In [ ]:
transfers = pd.read_excel('raw_data\\Subsidies and other transfers.xls',
                          na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(transfers.columns.to_list())

In [ ]:
transfers = filter_countries(transfers, 'Country Name')
transfers.head()

In [268]:
transfers.to_excel('preclean_data\\Subsidies and other transfers.xlsx', index=False)

In [ ]:
urban = pd.read_excel('raw_data\\Urban population (% of total population).xls',
                      na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(urban.columns.to_list())

In [ ]:
urban = filter_countries(urban, 'Country Name')
urban.head()

In [271]:
urban.to_excel('preclean_data\\Urban population (% of total population).xlsx', index=False)

In [ ]:
precip = pd.read_excel('raw_data\\Average precipitation in depth (mm per year).xls',
                       na_values=['..', '…', '', 'NULL', 'null', 'NA', 'n/a', '#N/A'])
print(precip.columns.to_list())

In [ ]:
precip = filter_countries(precip, 'Country Name')
precip.head()

In [274]:
precip.to_excel('preclean_data\\Average precipitation in depth (mm per year).xlsx', index=False)

In [ ]:
lpi_2010 = pd.read_excel('raw_data\\Logistic performance index.xlsx', sheet_name='2010')
print(lpi_2010.columns.to_list())

In [ ]:
lpi_2010 = lpi_2010[['Country', 'score']]
lpi_2010.shape

In [ ]:
lpi_2012 = pd.read_excel('raw_data\\Logistic performance index.xlsx', sheet_name='2012')
print(lpi_2012.columns.to_list())

In [282]:
lpi_2012 = lpi_2012[['Country', 'score']]
lpi_2012.shape

(155, 2)

In [ ]:
lpi_2014 = pd.read_excel('raw_data\\Logistic performance index.xlsx', sheet_name='2014')
print(lpi_2014.columns.to_list())

In [ ]:
lpi_2014 = lpi_2014[['Country', 'score']]
lpi_2014.shape

In [ ]:
lpi_2016 = pd.read_excel('raw_data\\Logistic performance index.xlsx', sheet_name='2016')
print(lpi_2016.columns.to_list())

In [ ]:
lpi_2016 = lpi_2016[['Country', 'score']]
lpi_2016.shape

In [ ]:
lpi_2018 = pd.read_excel('raw_data\\Logistic performance index.xlsx', sheet_name='2018')
print(lpi_2018.columns.to_list())

In [ ]:
lpi_2018 = lpi_2018[['Country', 'score']]
lpi_2018.shape

In [ ]:
lpi_2023 = pd.read_excel('raw_data\\Logistic performance index.xlsx', sheet_name='2023')
print(lpi_2023.columns.to_list())

In [ ]:
lpi_2023 = lpi_2023[['Economy', 'LPI Score']]
lpi_2023.shape